In [1]:
import altair as alt
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('sqss_data.csv')
df['insert_date'] = pd.to_datetime(df['insert_date'])
df['hour'] = df['insert_date'].dt.hour
df = df.groupby(by='hour').mean().reset_index()
df = df.melt(id_vars=["hour"], 
        var_name="category", 
        value_name="value")
df1 = df.loc[df['category'] == 'co2x' ]
df2 = df.loc[df['category'] == 'o2xx' ]

In [10]:
def plot_data(df):
    # Create a selection that chooses the nearest point & selects based on x-value
    nearest = alt.selection(type='single', nearest=True, on='mouseover',
                            fields=['hour'], empty='none')

    # The basic line
    line = alt.Chart(df).mark_line(interpolate='basis').encode(
        x='hour:Q',
        y='value:Q',
        color='category:N'
    )

    # Transparent selectors across the chart. This is what tells us
    # the x-value of the cursor
    selectors = alt.Chart(df).mark_point().encode(
        x='hour:Q',
        opacity=alt.value(0),
    ).add_selection(
        nearest
    )

    # Draw points on the line, and highlight based on selection
    points = line.mark_point().encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0))
    )

    # Draw text labels near the points, and highlight based on selection
    text = line.mark_text(align='left', dx=5, dy=-5).encode(
        text=alt.condition(nearest, 'value:Q', alt.value(' '))
    )

    # Draw a rule at the location of the selection
    rules = alt.Chart(df).mark_rule(color='gray').encode(
        x='hour:Q',
    ).transform_filter(
        nearest
    )

    # Put the five layers into a chart and bind the data
    layer = alt.layer(
        line, selectors, points, rules, text
    ).properties(
        width=600, height=300
    )
    return (layer).interactive()
plot_data(df)

alt.LayerChart(...)

In [11]:
df

,hour,category,value
0,0,co2x,1049.212200
1,1,co2x,1141.784167
2,2,co2x,1073.403333
3,3,co2x,998.604442
4,4,co2x,971.253845
...,...,...,...
91,19,humi,59.751002
92,20,humi,62.136459
93,21,humi,62.745529
94,22,humi,64.011511
